In [1]:
ls

GoogleNews-vectors-negative300.bin
command.sh
floyd_requirements.txt
glove.42B.300d.txt
layer1rnnGPU.ipynb
layer1rnnGPUbig.ipynb
lstm_300_200_0.30_0.30.h5
test_spacyLemma.csv
test_spacyLemma_trigram.csv
train_spacyLemma.csv
train_spacyLemma_trigram.csv
x_final_features_MyMagic3_spacyLemma_trigram.csv


In [2]:
#!pip install nltk

In [3]:
#!pip install spacy
#from spacy.en import English
#nlp = English()

In [4]:
#!wget https://dl.dropboxusercontent.com/s/w5qxo2wy7vqbpra/test_spacyLemma_trigram.csv

In [5]:
#!wget https://dl.dropboxusercontent.com/s/tw53gghntsnjzet/train_spacyLemma_trigram.csv

In [6]:
#!wget https://dl.dropboxusercontent.com/s/py0sja1y2m1wtpp/x_final_features_MyMagic3_spacyLemma_trigram.csv.gz

In [7]:
#!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

In [8]:
#!unzip glove.42B.300d.zip

In [9]:
#!gunzip x_final_features_MyMagic3_spacyLemma_trigram.csv.gz

In [10]:
#!rm glove.840B.300d.zip 

In [11]:
#!rm GoogleNews-vectors-negative300.zip

In [12]:
#!rm glove.42B.300d.zip

In [13]:
#!rm x_final_features_MyMagic3_spacyLemma_trigram.csv.gz

In [14]:
ls -l --block-size=M

total 11221M
-rw-r--r-- 1 root root 3476M Dec 11  2013 GoogleNews-vectors-negative300.bin
-rw-r--r-- 1 root root    1M May 25 15:07 command.sh
-rw-r--r-- 1 root root    1M May 25 15:07 floyd_requirements.txt
-rw-rw-r-- 1 root root 4793M Oct 24  2015 glove.42B.300d.txt
-rw-r--r-- 1 root root    1M May 27 03:50 layer1rnnGPU.ipynb
-rw-r--r-- 1 root root    1M May 27 03:57 layer1rnnGPUbig.ipynb
-rw-r--r-- 1 root root  136M May 27 03:03 lstm_300_200_0.30_0.30.h5
-rw-r--r-- 1 root root  306M May 25 15:18 test_spacyLemma.csv
-rw-r--r-- 1 root root  294M May 27 00:54 test_spacyLemma_trigram.csv
-rw-r--r-- 1 root root   59M May 25 15:18 train_spacyLemma.csv
-rw-r--r-- 1 root root   56M May 27 00:54 train_spacyLemma_trigram.csv
-rw-r--r-- 1 root root 2105M May 27 00:54 x_final_features_MyMagic3_spacyLemma_trigram.csv


In [15]:
!free

              total        used        free      shared  buff/cache   available
Mem:       62879864     8799484    36735780      538060    17344600    53005284
Swap:             0           0           0


In [16]:
#==============================================================================
# New Lystdo LSTM    https://www.kaggle.com/lystdo/lb-0-18-lstm-with-glove-and-magic-features
#==============================================================================
########################################
## import packages
########################################
import csv
import codecs
import numpy as np
import pandas as pd

from collections import defaultdict

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [17]:
########################################
## set directories and parameters
########################################
BASE_DIR = ''
EMBEDDING_FILE = BASE_DIR + 'glove.42B.300d.txt'
TRAIN_DATA_FILE = BASE_DIR + 'train_spacyLemma_trigram.csv'
TEST_DATA_FILE = BASE_DIR + 'test_spacyLemma_trigram.csv'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = 300
num_dense = 300
rate_drop_lstm = 0.3
rate_drop_dense = 0.3

act = 'relu'
re_weight = True # whether to re-weight classes to fit the 17.5% share in test set

STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

In [18]:
########################################
## index word vectors
########################################
print('Indexing word vectors')

embeddings_index = {}
f = open(EMBEDDING_FILE, encoding="utf-8")
count = 0
for line in f:
    if count == 0:
        count = 1
        continue
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %d word vectors of glove.' % len(embeddings_index))

Indexing word vectors
Found 1917493 word vectors of glove.


In [19]:

# The function "text_to_wordlist" is from
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    if text == "":
        text = "-empty-"
    return(text)

texts_1 = [] 
texts_2 = []
labels = []
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts_1.append(text_to_wordlist(values[3]))
        texts_2.append(text_to_wordlist(values[4]))
        labels.append(int(values[5]))
print('Found %s texts in train.csv' % len(texts_1))

test_texts_1 = []
test_texts_2 = []
test_ids = []
with codecs.open(TEST_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        test_texts_1.append(text_to_wordlist(values[1]))
        test_texts_2.append(text_to_wordlist(values[2]))
        test_ids.append(values[0])
print('Found %s texts in test.csv' % len(test_texts_1))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_1 + texts_2 + test_texts_1 + test_texts_2)

sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
test_ids = np.array(test_ids)


Found 404290 texts in train.csv
Found 2345796 texts in test.csv
Found 116784 unique tokens
Shape of data tensor: (404290, 30)
Shape of label tensor: (404290,)


In [20]:
########################################
## generate leaky features
########################################

laptop = 0
if laptop == 1:
    drive = "C"
else:
    drive = "F"

preprocessing = "_MyMagic3_spacyLemma_trigram"


import numpy as np
import pandas as pd
import operator
from sklearn.model_selection import train_test_split
from collections import Counter
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from pylab import plot, show, subplot, specgram, imshow, savefig
from sklearn.model_selection import KFold
K = 5
kf = KFold(n_splits = K)
#==============================================================================
# print(list(enumerate(kf.split([1,2,3,4,5,6,7,8,9]))))   
# X= np.array(["a","b","c","d","e","f","g","h","i"])
# for kth, (train_index, test_index) in enumerate(kf.split(X)):
#     print(test_index)
#     max = np.amax(test_index)+1
#     min = np.amin(test_index)
#     print(X[min:max])
#==============================================================================

np.set_printoptions(threshold=400000)
pd.set_option('display.max_rows', 2000, 'display.max_columns', 2000,  'display.show_dimensions', 'truncate')


RS = 12357
np.random.seed(RS)

print("Loading data")
input_folder = ''
df_train = pd.read_csv("train_spacyLemma_trigram.csv")
df_test  = pd.read_csv("test_spacyLemma_trigram.csv")
df_test.drop(["question1", "question2"], axis=1, inplace=True)

#adding final x features
print("Loading x final features")
#x = pd.read_csv(drive + ':/DS-main/Kaggle-main/Quora Question Pairs - inputs/data/x_final_features.csv', header=0) 
#x = pd.read_csv(drive + ':/DS-main/Kaggle-main/Quora Question Pairs - inputs/data/x_final_features_expended.csv', header=0) 
#x = pd.read_csv(drive + ':/DS-main/Kaggle-main/Quora Question Pairs - inputs/data/x_final_features_lowRemoved.csv', header=0) 
x = pd.read_csv('x_final_features' + preprocessing  + '.csv', header=0) 
x = x.drop(["rarestWordID1", "rarestWordID2", 'max_kcore', "tfidf_word_match", "wordMatchShare"], axis=1)

print(x.columns)
print(x.describe())

feature_names = list(x.columns.values)
print("Features: {}".format(feature_names))

x_train = x[:df_train.shape[0]].astype("float64")
x_test_real  = x[df_train.shape[0]:].astype("float64")
labels = df_train['is_duplicate'].values
x_train_real =  x_train
labels_real =  labels
del x, df_train




####################################################################
# preprocessing Imputer
####################################################################


from sklearn.preprocessing import Imputer
x_train_real = Imputer().fit_transform(x_train_real)
x_test_real = Imputer().fit_transform(x_test_real)







####################################################################
# preprocessing StandardScaler
####################################################################

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(np.vstack((x_train_real, x_test_real)))
x_train_real = ss.transform(x_train_real)
x_test_real = ss.transform(x_test_real)


Loading data
Loading x final features
Index(['word_match', 'word_match_2root', 'shared_count', 'stops1_ratio',
       'stops2_ratio', 'shared_2gram', 'cosine', 'words_hamming',
       'diff_stops_r', 'len_q1', 'len_q2', 'diff_len', 'caps_count_q1',
       'caps_count_q2', 'diff_caps', 'len_char_q1', 'len_char_q2',
       'diff_len_char', 'len_word_q1', 'len_word_q2', 'diff_len_word',
       'avg_world_len1', 'avg_world_len2', 'diff_avg_word', 'exactly_same',
       'duplicated', 'q1_how', 'q2_how', 'how_both', 'q1_what', 'q2_what',
       'what_both', 'q1_which', 'q2_which', 'which_both', 'q1_who', 'q2_who',
       'who_both', 'q1_where', 'q2_where', 'where_both', 'q1_when', 'q2_when',
       'when_both', 'q1_why', 'q2_why', 'why_both', 'q1_freq', 'q2_freq',
       'common_words', 'fuzz_qratio', 'fuzz_WRatio', 'fuzz_partial_ratio',
       'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio',
       'fuzz_token_set_ratio', 'fuzz_token_sort_ratio', 'wmd', 'norm_wmd',
       'c

In [21]:
########################################
## prepare embeddings
########################################
print('Preparing embedding matrix')

nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 26079


In [22]:
########################################
## sample train/validation data
########################################
#np.random.seed(1234)
perm = np.random.permutation(len(data_1))
idx_train = perm[:int(len(data_1)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data_1)*(1-VALIDATION_SPLIT)):]

data_1_train = np.vstack((data_1[idx_train], data_2[idx_train]))
data_2_train = np.vstack((data_2[idx_train], data_1[idx_train]))
x_train_real_train = np.vstack((x_train_real[idx_train], x_train_real[idx_train]))
labels_train = np.concatenate((labels[idx_train], labels[idx_train]))

data_1_val = np.vstack((data_1[idx_val], data_2[idx_val]))
data_2_val = np.vstack((data_2[idx_val], data_1[idx_val]))
x_train_real_val = np.vstack((x_train_real[idx_val], x_train_real[idx_val]))
labels_val = np.concatenate((labels[idx_val], labels[idx_val]))

weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.472001959
    weight_val[labels_val==0] = 1.309028344

In [23]:
########################################
## define the model structure
########################################
embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)
#lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)
lstm_layer0 = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm, return_sequences=True)
lstm_layer1 = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer0(embedded_sequences_1)
x1 = lstm_layer0(x1)
x1 = lstm_layer1(x1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer0(embedded_sequences_2)
y1 = lstm_layer0(y1)
y1 = lstm_layer1(y1)

x_train_real_input = Input(shape=(x_train_real.shape[1],))
x_train_real_dense = Dense(num_dense, activation=act)(x_train_real_input)
x_train_real_dense = Dropout(rate_drop_dense)(x_train_real_dense)
x_train_real_dense = BatchNormalization()(x_train_real_dense)
x_train_real_dense = Dense(num_dense, activation=act)(x_train_real_dense)

merged = concatenate([x1, y1, x_train_real_dense])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = concatenate([x1, y1, x_train_real_dense])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = concatenate([x1, y1, x_train_real_dense])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

########################################
## add class weight
########################################
if re_weight:
    class_weight = {0: 1.309028344, 1: 0.472001959}
else:
    class_weight = None

ResourceExhaustedError: OOM when allocating tensor with shape[116785,300]
	 [[Node: embedding_1/embeddings/Assign = Assign[T=DT_FLOAT, _class=["loc:@embedding_1/embeddings"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](embedding_1/embeddings, embedding_1/random_uniform)]]

Caused by op 'embedding_1/embeddings/Assign', defined at:
  File "/usr/local/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/site-packages/zmq/

In [ ]:
print("kernel active")

In [ ]:

########################################
## train the model
########################################
model = Model(inputs=[sequence_1_input, sequence_2_input, x_train_real_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
#model.summary()
print(STAMP)

early_stopping =EarlyStopping(monitor='val_loss', patience=20)
bst_model_path = STAMP + '_big.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True, verbose=1)
batch_size = 512
hist = model.fit([data_1_train, data_2_train, x_train_real_train], labels_train, \
        validation_data=([data_1_val, data_2_val, x_train_real_val], labels_val, weight_val), \
        epochs=2000, batch_size=batch_size, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint], verbose=2)

model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_loss'])

########################################
## make the submission
########################################
print('Start making the submission before fine-tuning')

preds = model.predict([test_data_1, test_data_2, x_test_real], batch_size=batch_size, verbose=2)
preds += model.predict([test_data_2, test_data_1, x_test_real], batch_size=batch_size, verbose=2)
preds /= 2

submission = pd.DataFrame({'test_id':test_ids, 'is_duplicate':preds.ravel()})
submission.to_csv('newlystdo_%.4f_'%(bst_val_score)+STAMP+'_madeBig.csv', index=False)




lstm_300_300_0.30_0.30
Train on 727722 samples, validate on 80858 samples
Epoch 1/2000
Epoch 00000: val_loss improved from inf to 0.19061, saving model to lstm_300_300_0.30_0.30.h5
981s - loss: 0.2276 - acc: 0.8321 - val_loss: 0.1906 - val_acc: 0.8580
Epoch 2/2000
